## 5.- Webserver con ESP8266

En este capitulo se describira como controlar salidas tales como: Motores, Led, servomotores, etc. Entradas tales como: Sensores de temperatura, humedad, distancia, etc. Para cumplir con este fin se programara al ESP8266 como un servidor que permita realizar las acciones. Este programa se denominara webserver.

El programa para el ESP8266 debe:
- Conectarse a Internet.
- Controlar la salidad.
- Programar un webserver que realice las acciones.

Para conectar el ESP8266 a una red WIFI se debe utilizar la libreria  ESP8266Wifi y para construir el webServer la libreria ESP8266WebServer, para acceder a la memoria FLASH se utilizara PROGMEN

### 5.1.- Libreria Esp8266Wifi

La librería WiFi para ESP8266  ha sido desarrollada para, conectar el soc (sistema en un chip) a una red WIFI. basándose en el SDK de ESP8266, usando nombres convencionales y la filosofía de funcionalidades generales de la librería WiFi de Arduino. 

|Nombre  |   Sintaxis                  |            Funcion                                        |
|---     |---                          |---                                                        |
|begin   |WiFi.begin(ssid,contraseña); |Constructor del objeto  WIFI                               |
|        |                             |ssid: nombre de la red WIFI                                |
|        |                             |contraseña: contraseña de la red                           |
|status  |WiFi.status()                | Presenta el estado de la conexion                         |
|        |                             |  WL_CONNECTED: indica que esta conectado                  |
|localIP |WiFi.localIP()               | Presenta la direccion de red donde se conecto             |


### 5.2- Codigo de prueba ESP8266WiFi

Se presenta el codigo de un programa que permite conectar un ESP8266 a una red. Se utiliza las funciones descriptas en la seccion anterior.
```
#include <ESP8266WiFi.h>  
void setup()  
{ Serial.begin(115200);  
Serial.println();  
WiFi.begin("nombre-red", "contraseña-red");  
Serial.print("Conectando");  
while (WiFi.status() != WL_CONNECTED)  
{  
delay(500);  
Serial.print(".");  
}  
Serial.println();  
Serial.print("Conectado, dirección IP: ");  
Serial.println(WiFi.localIP());  
}  

void loop() {}
``` 


Para poder realizar un codigo de programa reusable y facilmente modificable. Se recomienda crear tres archivos.

- internet0.ino: Donde se encuentra el programa principal.
- config.h: Donde se encuentran todas las constantes del programa.
- wifi.hpp: Donde se encuentran las funciones que se utizan para trabajae en Internet.

#### Abrir el IDE de Arduino.
  Seleccionar  Archivo -> New Sketch apretar ENTER   
  
  Aparece un sketch nuevo como el de la figura.   
  
<center><img src="img/wifi0.jpg" alt="Imagen"></img></center>  
  Seleccionar Archivo -> Save   
  
En la pestaña que se muestra en la figura, se coloca internet0 en el dialogo Nombre y  apretar el boton Guardar   

<center><img src="img/wifi1.jpg" alt="Imagen"></img></center>   

Seleccionar con un click del raton en los tres puntos.   

<center><img src="img/wifi2.jpg" alt="Imagen"></img></center>  

Aparece el dialogo que se muestra en la figura.   

<center><img src="img/wifi3.jpg" alt="Imagen"></img></center>

Seleccionar nueva pestaña, en la misma colocar config.h, repitiendo este procedimiento colocar wifi.hpp. El resultado se muestra en la siguiente figura.   

<center><img src="img/wifi4.jpg" alt="Imagen"></img></center>  


Se debe colocar en config.h el siguiente codigo
```
const char* red_WIFI = "nombre_red";
const char* clave = "clave";
```

En wifi.hpp se debe escribir el siguiente codigo
```
void conectarWIFI() {
  WiFi.begin(red_WIFI, clave);
  Serial.print("Conectando");
  while (WiFi.status() != WL_CONNECTED) {
    delay(500);
    Serial.print(".");
  }
  Serial.println();
  Serial.print("Conectado, dirección IP: ");
  Serial.println(WiFi.localIP());
}
```

En internet0.ino se debe escribir el siguiente codigo.

```
#include <ESP8266WiFi.h>
#include "config.h"
#include "wifi.hpp"
void setup()  
{
Serial.begin(115200);  
conectar_WIFI();
}  

void loop() {}
``` 
Se debe compilar y cargar el programa en el esp8266. Cuando se ejecuta, aparece en la consola, la dirección de IP, donde se conecto el dispositivo.


## 5.3.- Servidor WEB

En esta seccion se describira como implemetar un servidor web en el SOC ESP8266 que permita, en este caso prender o apagar un led. Se utilizara una libreria especialmente desarrollada con este fin.
### 5.3.1 Librería ESP8266WebServer
 
Un servidor web es una computadora que almacena los archivos que componen un sitio web (ej.  documentos HTML , imágenes, hojas de estilos CSS y archivo JavaScript) y los entrega a otro dispositivo que le solicita un servicio. Se desarrollo para el ESP8266 una libreria que permite implementar un servidor web. Se denomina libreria esp8266WebServer. Los metodos que se utilizan se muestran a continuacion

| Metodo          | Sintaxis                   |  Funcion      |
|---              |---                         |---            |
|ESP8266WebServer |ESP8266WebServer server(80) | Crea el objeto server |
|begin            | server.begin()             | Inicializa el servidor|
|close            | server.close()             | Apaga el servidor     |
|stop             | server.stop()              | Apaga el servidor     |
| on              | server.on("/", FuncionM);  | Indica una función que emitira una respuesta del servidor cuando un navegador solicita la URL |
|addHandler       | server. addHandler();      |              |
| onNotFound      | server.onNotFound();       |              |
| onFileUpload    | server.onFileUpload();     |              |
| send            | server.send(code,tipo_contenido,contenido)| Envia las respuestas solicitadas code:200, 404 tipo_contenido:"text/plain","image/png"|
|streamFile      |server.streamFile(file, GetContentType(path));| Permite cargar una pagina completa |
|handleClient     |server.handleClient();      |  Maneja las conexiones entrantes|


### 5.3.2 PROGMEM

Permite cargar informacion en la memoria flash. Despues se puede pasar a la RAM cuando hace falta. Es una característica Arduino AVR que ha sido portada a ESP8266 para asegurar la compatibilidad con las librerías existentes en Arduino, así como para ahorrar RAM. La sintaxis seria:

``` const char MAIN_page[] PROGMEM = R"=====( pagina web )======" ```


### 5.3.3 Programa Webserver
Este programa debe prender o apagar el led que se encuentra en la placa desde una pagina WEB. Este sistema puede ser controlado desde un telefono celular. Se utiliza el microcontrolador esp8266 que se conecta a la red WIFI y crea un webserver. Utliza la memoria RAM para guardar la pagina WEB.

Se crea un nuevo programa en el IDE de arduino denominado internet1.ino y los archivos config.h y wifi.hpp se copian en el nuevo directorio.


#### Archivo config.h 

En el archivo config.h se escribe la pagina WEB que se muestra mas abajo

   
<!DOCTYPE html>  
<html>  
<body>  
<center>
<h1> CURSO DE ROBOTICA EDUCATIVA</h1>
<h1>Programa para prender/apagar un LED </h1><br> <br>
Apretar para <a href="ledOn" >Prender LED </a><br>
Apretar para <a href="ledOff">Apagar LED</a><br>
<hr>
</center>

</body>
</html>   
El archivo config.h queda asi:      

```
const char* red_WIFI     = "nombre_red";
const char* clave = "clave";

const char MAIN_page[] PROGMEM = R"=====(  
<!DOCTYPE html>  
<html>  
<body>  
<center>
<h1> CURSO DE ROBOTICA EDUCATIVA</h1>
<h1>Programa para prender/apagar un LED </h1><br> <br>
Apretar para <a href="ledOn" ><button>Prender </button>LED</a><br>
Apretar para <a href="ledOff"><button>Apagar LED</button></a><br>
<hr>
</center>

</body>
</html>
)=====";
```

#### Archivo wifi.hpp

En esp8266 para configurar y controlar un webserver se usan las siguientes funciones.   
   
|  Funcion        |   Sintaxis                   |Acciones                   |   
|---              |---                           |---                        |   
|ESP8266WebServer |  ESP8266WebServer server(80) |Crea el objeto server. |     
| begin           |  server.begin()              |Inicia el servidor web, especificando el puerto (generalmente el 80). |   
| on              |  server.on(/ledOn,funcion    |Permiten definir rutas (URI) y las funciones que se ejecutarán cuando se acceda a esas rutas. |   
|handleClient     |  server.handleClient()       | Escucha las conexiones entrantes y gestiona las solicitudes. |   
|send             |server.send( 200,s)           |Se utilizan para enviar respuestas al cliente, incluyendo código HTML en el string s y OK con 200.  |   
En el archivo wifi.hpp se agregan tienen que estar las siguientes funciones:
- iniciar_server()
- controlRaiz()
- prender()
- apagar()
  

 ##### Funcion iniciar_server   
 Esta funcion es la encargada de enviar la informacion a la pagina web cuando es solicitada, asocia la ruta de la URL raíz ("/") con una función llamada controlRaiz. Cuando un cliente (por ejemplo, un navegador web) solicita la página principal del servidor (la dirección IP del ESP8266), la función controlRaiz se ejecutará para manejar esa solicitud

```
void iniciarServer() {

  server.on("/", controlRaiz);
  server.on("/Prender", prender);  //Rutina mueve el brazo
  server.on("/Apagar", apagar);    //Rutina mueve el brazo
  server.begin();
}

```


##### Funcion controlRaiz 

En este programa carga la pagina web que se encuentra en la memoria RAM y esta definida en el archivo config.h. Se muestra el codigo de la funcion.   
```
void controlRaiz() {  
 Serial.println("Pagina original ");  
 String s = MAIN_page; //Read HTML contents  
 server.send(200, "text/html", s); //Send web page  
}
```

##### Funcion prender 
Esta funcion prende el LED que se encuentra dentro de la placa esp8266.
```  
void prender() {   
 Serial.println("LED prendido");  
 digitalWrite(BUILTIN_LED,LOW); //LED is connected in reverse  
 server.send(200, "text/html", "El LED esta prendido");  
}
```

#####  Funcion apagar 
Funcion apaga el LED que se encuentra dentro de la placa esp8266.
```
void apagar() {   
 Serial.println("LED apagado");  
 digitalWrite(BUILTIN_LED,HIGH); //LED off  
 server.send(200, "text/html", "El LED esta apagado");   
}  
```

##### Archivo internet1.ino 

En este archivo deben estar las funciones setu() y loop se debe escribir el siguiente codigo:

```
#include <ESP8266WiFi.h>  
#include <WiFiClient.h>  
#include <ESP8266WebServer.h> 
ESP8266WebServer server(80); //Server on port 80  
#include <config.h>
#include <wifi.hpp>

//==============================================================  
//                  SETUP  
//==============================================================  
void setup(void){  
  Serial.begin(115200);  
  conectarWiFi();
iniciarServer();
}  
//==============================================================  
//                     LOOP  
//==============================================================  
void loop(void){  
  server.handleClient();          //Handle clien  t requests
```

Se debe compilar y transferir el programa. En la consola de la puerta serie aparece el numero de IP de la placa y colocando ese numero en el navegador se obtiene la pagina WEB, apretando los botones se puede observar en la placa que se prende o apaga el LED del esp8266.